In [1]:
import networkx as nx
import numpy as np
import pandas as pd
from IPython.display import set_matplotlib_formats
from monthly_graphs import ImportToDf, ReduceTimeInterval, split_by_month
from read_network import merge_edges
from structural_balance import is_balanced

In [2]:
%matplotlib inline
set_matplotlib_formats('svg')

In [3]:
title = ImportToDf('data/soc-redditHyperlinks-title.tsv', delimiter = '\t')
body = ImportToDf('data/soc-redditHyperlinks-body.tsv', delimiter = '\t')
hyperlinks = pd.concat([title, body])

In [4]:
# weighted monthly graphs
wmgs = split_by_month(hyperlinks, undirected=True, threshold=1/3)

In [5]:
balanced_months = []
largest_component_sizes = np.empty(len(wmgs))
for i, WMG in enumerate(wmgs):
    largest_component_sizes[i] = len(max(nx.connected_components(WMG), key=len))
    if is_balanced(WMG):
        balanced_months.append((i, WMG))
print(f'{len(balanced_months)} monthly graphs were balanced out of {len(wmgs)}')
print(f'The mean size of the largest connected components '
      f'in the monthly graphs is {largest_component_sizes.mean()}')

0 monthly graphs were balanced out of 40
The mean size of the largest connected components in the monthly graphs is 6767.6


In [6]:
# try with daily graphs
hyperlinks['TIMESTAMP'] = pd.to_datetime(hyperlinks['TIMESTAMP'])
gb = hyperlinks.groupby(hyperlinks['TIMESTAMP'].dt.floor('d'))
daily_links = [gb.get_group(g) for g in gb.groups]
daily_graphs = [nx.from_pandas_edgelist(links, source='SOURCE_SUBREDDIT',
                                        target='TARGET_SUBREDDIT',
                                        edge_attr='LINK_SENTIMENT',
                                        create_using=nx.MultiDiGraph)
               for links in daily_links]
# weighted daily graphs
wdgs = [merge_edges(graph, undirected=True, threshold=1/3) for graph in daily_graphs]

In [7]:
balanced_days = []
largest_component_sizes = np.empty(len(wdgs))
for i, WDG in enumerate(wdgs):
    largest_component_sizes[i] = len(max(nx.connected_components(WDG), key=len))
    if is_balanced(WDG):
        balanced_days.append((i, WDG))
print(f'{len(balanced_days)} daily graphs were balanced out of {len(wdgs)}')
print(f'The size of the largest connected components in the first of the '
      f'balanced graphs is {largest_component_sizes[balanced_days[0][0]]}')
print(f'The mean size of the largest connected components '
      f'in the daily graphs is {largest_component_sizes.mean():.2f}')

1 daily graphs were balanced out of 1217
The size of the largest connected components in the first of the balanced graphs is 4.0
The mean size of the largest connected components in the daily graphs is 347.88


In [8]:
# some tests for is_balanced, on an example network from the course book (section 5.5):
# https://www.cs.cornell.edu/home/kleinber/networks-book/
G = nx.Graph()
G.add_edges_from([(1, 2, {'SIGN': 1}), (1, 3, {'SIGN': 1}), (2, 3, {'SIGN': 1}),
                  (2, 5, {'SIGN': 1}), (6, 8, {'SIGN': 1}), (9, 12, {'SIGN': 1}),
                  (7, 12, {'SIGN': 1}), (10, 12, {'SIGN': 1}), (12, 13, {'SIGN': 1}),
                  (4, 2, {'SIGN': -1}), (4, 7, {'SIGN': -1}), (4, 9, {'SIGN': -1}),
                  (6, 3, {'SIGN': -1}), (6, 5, {'SIGN': -1}), (6, 11, {'SIGN': -1}),
                  (8, 11, {'SIGN': -1}), (11, 14, {'SIGN': -1}), (11, 10, {'SIGN': -1}),
                  (11, 13, {'SIGN': -1}), (15, 14, {'SIGN': -1}), (15, 13, {'SIGN': -1})])
print(not is_balanced(G))

# now modify a few edges so that it is balanced
nx.set_edge_attributes(G, values={(6, 11): 1, (8, 11): 1, (11, 14): 1, (15, 13): 1}, name='SIGN')

# now try to add three more connected components
G.add_edges_from([(42, 23, {'SIGN': 1}), (23, 17, {'SIGN': 1}), (17, 19, {'SIGN': 1}),
                  (77, 88, {'SIGN': 1}), (88, 99, {'SIGN': -1}), (77, 99, {'SIGN': -1}),
                  (55, 66, {'SIGN': -1})])
print(len(list(nx.connected_components(G))))
print(is_balanced(G))

# now make one component unbalanced
G[88][99]['SIGN'] = 1
print(not is_balanced(G))

True
4
True
True


In [9]:
# let's examine other structural properties of the network
MG = nx.from_pandas_edgelist(hyperlinks, 'SOURCE_SUBREDDIT', 'TARGET_SUBREDDIT',
                             edge_attr=None, create_using=nx.MultiDiGraph)

In [10]:
print(f'There are a total of {len(MG)} nodes\nand {len(MG.edges)} edges in the network')

There are a total of 67180 nodes
and 858488 edges in the network


In [11]:
# is there a giant component?
wccs = sorted(nx.weakly_connected_components(MG), key=len, reverse=True)
print('Number of weakly connected components:', len(wccs))
print('Top sizes of weakly connected components:')
for c in wccs[:7]:
    print(len(c))

Number of weakly connected components: 712
Top sizes of weakly connected components:
65648
9
8
7
7
6
6


In [12]:
# what about strongly connected
sccs = sorted(nx.strongly_connected_components(MG), key=len, reverse=True)
print('Number of strongly connected components:', len(sccs))
print('Top sizes of strongly connected components:')
for c in sccs[:7]:
    print(len(c))

Number of strongly connected components: 45564
Top sizes of strongly connected components:
21432
5
5
4
4
4
3
